In [ ]:
import os
# Finding the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Installing Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Setting Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Starting a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
# Downloading the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-11-18 00:39:58--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.6’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-11-18 00:39:58 (6.71 MB/s) - ‘postgresql-42.2.16.jar.6’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Capstone").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Loading Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://datacamp-group3-bucket.s3.us-east-2.amazonaws.com/vacancies_test.csv"

### Creating DataFrames to match SQL tables

In [ ]:
from pyspark.sql.functions import to_date
# Reading in the Review dataset as a DataFrame
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
df.show(30)

+--------+--------------------+----------+-------------+------+--------------------+--------+----+-------+-----+
|REF_DATE|            Location|JobDetails|   Statistics| VALUE|            NOC_Desc|NOC_Code|Year|Quarter|   ID|
+--------+--------------------+----------+-------------+------+--------------------+--------+----+-------+-----+
| 2015-04|Newfoundland and ...| Full-time|Job vacancies| 430.0|Business, finance...|       1|2015|      2|36393|
| 2015-04|Newfoundland and ...| Full-time|Job vacancies| 170.0|Natural and appli...|       2|2015|      2|36978|
| 2015-04|Newfoundland and ...| Full-time|Job vacancies| 265.0| Health occupations |       3|2015|      2|37196|
| 2015-04|Newfoundland and ...| Part-time|Job vacancies| 260.0| Health occupations |       3|2015|      2|37199|
| 2015-04|Newfoundland and ...| Full-time|Job vacancies| 130.0|Occupations in ed...|       4|2015|      2|37788|
| 2015-04|Newfoundland and ...| Full-time|Job vacancies| 955.0|Sales and service...|       6|201

In [ ]:
# number of rows in the dataframe
df.count()

2875

In [ ]:
# creating a copy to store dataframe
df_copy = df
df_copy.show()

+--------+--------------------+----------+-------------+-----+--------------------+--------+----+-------+-----+
|REF_DATE|            Location|JobDetails|   Statistics|VALUE|            NOC_Desc|NOC_Code|Year|Quarter|   ID|
+--------+--------------------+----------+-------------+-----+--------------------+--------+----+-------+-----+
| 2015-04|Newfoundland and ...| Full-time|Job vacancies|430.0|Business, finance...|       1|2015|      2|36393|
| 2015-04|Newfoundland and ...| Full-time|Job vacancies|170.0|Natural and appli...|       2|2015|      2|36978|
| 2015-04|Newfoundland and ...| Full-time|Job vacancies|265.0| Health occupations |       3|2015|      2|37196|
| 2015-04|Newfoundland and ...| Part-time|Job vacancies|260.0| Health occupations |       3|2015|      2|37199|
| 2015-04|Newfoundland and ...| Full-time|Job vacancies|130.0|Occupations in ed...|       4|2015|      2|37788|
| 2015-04|Newfoundland and ...| Full-time|Job vacancies|955.0|Sales and service...|       6|2015|      2

In [ ]:
# dropping rows that have no data
df.na.drop().count()

2826

In [ ]:
# Understanding column types to help with SQL schema
df.dtypes

[('REF_DATE', 'string'),
 ('Location', 'string'),
 ('JobDetails', 'string'),
 ('Statistics', 'string'),
 ('VALUE', 'double'),
 ('NOC_Desc', 'string'),
 ('NOC_Code', 'int'),
 ('Year', 'int'),
 ('Quarter', 'int'),
 ('ID', 'int')]

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://group3db.cueberfdee85.us-east-2.rds.amazonaws.com:5432/sample_db"
config = {"user":"postgres", 
          "password": "databootcampcapstone",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Writing review_id_df to table in RDS
df.write.jdbc(url=jdbc_url, table='vacancies', mode=mode, properties=config)